In [13]:
import json
import torch
from gensim.models import KeyedVectors
from gensim import matutils
import numpy as np

In [14]:
print('Loading Vocabulary and Vectors')
word2vec = KeyedVectors.load_word2vec_format(
    'data/word2vec/GoogleNews-vectors-negative300.bin', binary=True)
vocabulary = json.load(open('data/processed_data/processed_data.json', 'r'))

Loading Vocabulary and Vectors


In [51]:
scores = rouge.get_scores(
    generated_summary, reference_summary
)
print(scores)

tensor(1.6194, grad_fn=<MseLossBackward0>)

In [102]:
def word2vec_emb(sequence, word2vec, vocabulary):
    
    seq_tokens = []
    
    for key in sequence:
        try:
            seq_tokens.append(vocabulary['ind2word'][str(key.item())])
        except:
            seq_tokens.append(vocabulary['ind2word'][list(
                vocabulary['ind2word'].keys())[-1]])
    
    vectorized = []
    for word in seq_tokens:
        try:
            vectorized.append(word2vec[word])
        except:
            vectorized.append(np.zeros(300,))
    
    return np.array(vectorized)

In [103]:
def similarity_cosine(vec1, vec2):
    cosine_similarity = np.dot(matutils.unitvec(vec1), matutils.unitvec(vec2))
    print(cosine_similarity)
    print(spatial.distance.cosine(vec1, vec2))
    return cosine_similarity

In [104]:
def reward(target, generated, word2vec, vocabulary):
    '''
    Calculate the reward for the generated text
    '''
    target = word2vec_emb(target, word2vec, vocabulary)
    generated = word2vec_emb(generated, word2vec, vocabulary)
    
    reward = similarity_cosine(target.mean(axis=0), generated.mean(axis=0))

    return reward

In [123]:
answer = torch.tensor([[6100, 4948, 6066, 1336, 4264, 4093,   37, 3343, 2467, 5527, 3396, 4326,
          219, 5560, 5845, 1464, 3149, 2805, 4154, 1899, 5642]])

question = torch.tensor([[6100, 2440, 5128, 5570,  511, 2483, 4828, 3564,  282, 3869, 2614,    7,
         4783, 4595, 5613, 1385, 6032, 4369,  286, 2259, 1900]])

summary = torch.tensor([[6100, 5955, 2642, 6021, 1982, 3081, 3554, 3802, 4577, 5298, 5555, 1110,
         4515, 2661, 2299, 4606, 4956, 4007, 3281, 2625, 2206, 4335, 2730, 2925,
         4523, 1231, 4269,  427,  861, 4104, 2630, 2499, 5430, 4043, 5146, 4595,
         3772, 5545, 4852, 3325, 4324]])

gtSummary = torch.tensor([[405,  333,  880,  188,  624,  194,  996,  293,  244, 6099, 3275,   99,
           25,  244,  754,  636,  936, 1461, 3653,   51,   55,   56,   57,    1,
           58,    1,   59,    1,   60,   52,  244, 1367, 6099, 1368, 3654, 2424,
           76,   51, 3655,   52]])

In [159]:
from scipy import spatial

In [128]:
# torch.cat((answer[0], question[0]))
print(reward(target=torch.cat((answer[0], question[0])), generated=summary[0], word2vec=word2vec, vocabulary=vocabulary))
print(reward(target=gtSummary[0], generated=summary[0], word2vec=word2vec, vocabulary=vocabulary))

0.7565139467498903
0.6523351061715779


In [147]:
from rouge import Rouge
def rouge_scores(target, generated, word2vec, vocabulary):
    rouge = Rouge()
    tar = []
    ref = []

    for key in target:
        try:
            if vocabulary['ind2word'][str(key.item())] !='UNK':
                tar.append(vocabulary['ind2word'][str(key.item())])
        except:
            pass

    for key in generated:
        try:
            if vocabulary['ind2word'][str(key.item())] !='UNK':
                ref.append(vocabulary['ind2word'][str(key.item())])
        except:
            pass

    return rouge.get_scores(' '.join(ref), ' '.join(tar), avg=True)

In [158]:
rouge_scores(torch.cat((answer[0], question[0])), summary[0], word2vec, vocabulary)

40 visible one-half calling retinal open-angle missing elbows street hopeless absolutely S loss Depakene TEE acid minimal double recurrences opposite Cutting tends efforts settings hepatitis reflexes heart-lung Name 1 Oxygen endoscopy if Autonomic beat companies cervical Nonne-Milroy separates Fragile osteochondrodysplasias abnormally
- - - - - - 
40 running electrical Recurrent barley amyloid Rapid senses containers Evidence Hospitalization testing Joseph FODMAP x-rays fasting coated Southeast psoralen Sharing natural G pacing mouthwashes Rasagiline While psychiatric resistance new lights relieving babies Involved specially firm beat relationships instability cookies colloidal mechanisms


{'rouge-1': {'r': 0.025, 'p': 0.025, 'f': 0.024999995000001003},
 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.025, 'p': 0.025, 'f': 0.024999995000001003}}

In [138]:
Rouge().get_scores('I go to the beach','I go to the beach every data', avg=True)

{'rouge-1': {'r': 0.7142857142857143, 'p': 1.0, 'f': 0.8333333284722222},
 'rouge-2': {'r': 0.6666666666666666, 'p': 1.0, 'f': 0.7999999952000001},
 'rouge-l': {'r': 0.7142857142857143, 'p': 1.0, 'f': 0.8333333284722222}}